# Compression Notebook
This notebook exists to compress the cfpb csv, which is too large for github.

In [1]:
import pandas as pd
import numpy as np

import time
import os

In [4]:
PATH_PARENT = os.path.dirname(os.getcwd())
PATH_RAW = PATH_PARENT + "\\data_raw\\"
PATH_PROC = PATH_PARENT + "\\data_processed\\"

In [30]:
cc_raw_df = pd.read_csv(PATH_RAW+"Consumer_Complaints_20190324.csv"
                        , dtype={"Product": "category", "Sub-product": "category", "Issue": "category"
                              , "Sub-issue": "category", "Company public response": "category"
                              , "Consumer consent provided?": "category", "Consumer disputed?": "category"
                              , "Submitted via": "category", "Tags": "category"
                              , "State": "category", "Timely response?": "category"                                
                                }
                       )

C:\ProgramData\Anaconda3\envs\cfpb-kerasgpu-env\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
cc_proc_df = cc_raw_df.copy()
cc_proc_df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/22/2019,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,NaN,NaN,OCWEN LOAN SERVICING LLC,VA,221XX,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3187532
1,03/22/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,USCB Corporation,MD,207XX,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3188247
2,03/22/2019,Vehicle loan or lease,Loan,Struggling to pay your loan,Loan balance remaining after the vehicle is re...,NaN,Company has responded to the consumer and the ...,"First Investors Financial Services Group, Inc.",IL,NaN,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3187579
3,03/22/2019,"Money transfer, virtual currency, or money ser...",International money transfer,Other transaction problem,NaN,NaN,NaN,RIA FINANCIAL,CA,90280,NaN,NaN,Phone,03/22/2019,In progress,Yes,NaN,3187502
4,03/22/2019,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,ALLY FINANCIAL INC.,FL,33162,NaN,NaN,Web,03/22/2019,In progress,Yes,NaN,3187649


In [32]:
# d.memory_usage()
# d["Complaint.ID"].fillna(0)
# cc_raw_df['ID'] = cc_raw_df['ID'].fillna(0).astype(np.int32)
cc_proc_df['Complaint ID'] = cc_proc_df['Complaint ID'].fillna(0).astype(np.int32)

In [33]:
cc_raw_df.memory_usage()

Index                               128
Date received                   9956664
Product                         1245283
Sub-product                     1247295
Issue                           2494670
Sub-issue                       2499206
Consumer complaint narrative    9956664
Company public response         1244963
Company                         9956664
State                           1247191
ZIP code                        9956664
Tags                            1244715
Consumer consent provided?      1244787
Submitted via                   1244803
Date sent to company            9956664
Company response to consumer    9956664
Timely response?                1244707
Consumer disputed?              1244707
Complaint ID                    9956664
dtype: int64

In [34]:
cc_proc_df.memory_usage()

Index                               128
Date received                   9956664
Product                         1245283
Sub-product                     1247295
Issue                           2494670
Sub-issue                       2499206
Consumer complaint narrative    9956664
Company public response         1244963
Company                         9956664
State                           1247191
ZIP code                        9956664
Tags                            1244715
Consumer consent provided?      1244787
Submitted via                   1244803
Date sent to company            9956664
Company response to consumer    9956664
Timely response?                1244707
Consumer disputed?              1244707
Complaint ID                    4978332
dtype: int64

In [40]:
cc_proc_df.to_pickle(PATH_PROC+"Consumer_Complaints_20190324_2.pkl", compression='zip')

In [42]:
cc_proc_test = pd.read_pickle(PATH_PROC+"Consumer_Complaints_20190324_2.pkl", compression='zip')
cc_proc_test.shape

(1244583, 18)

In [46]:
"""
FUNCTIONS
"""

def createSpreadsheets(dataframe, version, chunk_size):
    if len(dataframe) >  chunk_size:
        remainder = len(dataframe)%chunk_size
        matters_create_remainder = dataframe.iloc[0:remainder+1]
        matters_create_chunkable = dataframe.iloc[remainder:]
    
        # Write matters_create_remainder to data_out
        matters_create_remainder.to_pickle(PATH_PROC+'/Consumer_Complaints_'+version+'_00.pkl', compression='zip')
    
        # Write matters_Create_chunkable to data_out
        chunksize = chunk_size
        i = 1
        for chunk in np.split(matters_create_chunkable,len(matters_create_chunkable) // chunksize):
            chunk.to_pickle(PATH_PROC+'/Consumer_Complaints_'+version+'_{:02d}.pkl'.format(i), compression='zip')
            i += 1
    else:
        # Write update matters to data-out
        matters_update.to_pickle(PATH_PROC+'/Consumer_Complaints_'+version+'_00.pkl', compression='zip')
        
        # Write matters_create_remainder to data_out
        dataframe.to_pickle(PATH_PROC+'/Consumer_Complaints_'+version+'_00.pkl', compression='zip')


In [47]:
createSpreadsheets(cc_proc_df,  version='20190324', chunk_size=100000)